## 使用OpenAI文本嵌入

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from illufly.knowledge import LocalFilesLoader

In [ ]:
qa = LocalFilesLoader(["QA", "工美港", "汇报资料"])
# qa.load()
qa.get_files()

In [ ]:
model = OpenAIEmbeddings()
qa.cache_embeddings(model, 'openai')

In [ ]:
from langchain_community.vectorstores import FAISS
from illufly.memory import MemoryManager
from illufly.knowledge import collect_docs
from illufly.chain import create_qa_chain
from langchain_core.runnables import chain

db = FAISS.from_embeddings(*qa.load_embeddings(model, 'openai'))
retriever = db.as_retriever()

@chain
def get_retriever(input, config):
    session_id = config['configurable'].get('session_id', 'default')
    print(input, session_id)
    return db.as_retriever()
    
memory = MemoryManager()
chain = create_qa_chain(ChatOpenAI(), get_retriever, memory)

In [ ]:
for x in chain.stream({"input": "工美港有多少正常的消防阀门？"}):
    print(x.content, end="")

In [ ]:
for x in chain.stream({"input": "辣子鸡怎么做？"}):
    print(x.content, end="")

In [ ]:
chain.get_prompts()[0]

## 使用阿里云文本嵌入

In [13]:
from typing import List, Any
from illufly.chain import create_qa_chain
from illufly.memory import MemoryManager
from illufly.knowledge import LocalFilesLoader

from langchain_core.runnables import chain, Runnable
from langchain_community.llms import Tongyi
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import DashScopeEmbeddings

llm = Tongyi(model='qwen2-7b-instruct')
vectorDB = FAISS
embeddings = DashScopeEmbeddings()
embeddings_tag = "qwen"

qa_folders = ["QA", "工美港", "汇报资料"]
base_folder = ""

# 实现动态的文本嵌入加工
@chain
def get_retriever(input, config):
    qa = LocalFilesLoader(qa_folders, base_folder)
    qa.cache_embeddings(embeddings, embeddings_tag)

    resp = qa.load_embeddings(embeddings, embeddings_tag)
    return vectorDB.from_embeddings(*resp).as_retriever()

memory = MemoryManager()
chain = create_qa_chain(llm, get_retriever, memory, prompt_id="RAG-XIAOFANG")

In [14]:
for x in chain.stream({"input": "工美港有多少正常的消防阀门？"}):
    print(x, end="")

问题答案：工美港有300个正常的消防阀门。相关规范解释：这一信息来自于对工美港基本资料的描述，提到在320个消防阀门中，有300个处于正常水压状态，剩余的20个阀门虽然压力略高但仍在可控范围内。